In [50]:
import os
import numpy as np
import pandas as pd
import datetime
import requests
import pickle
from zipfile import ZipFile
from io import BytesIO, StringIO
from csv import reader
from IPython.display import clear_output
from collections import defaultdict

In [2]:
years = range(2013, 2020)
months = range(1, 13)
year_months = [str(y) + str(m).zfill(2) for y in years for m in months]
# exclude months before citibike started and in the future
year_months = year_months[5:-4]
url_start = "https://s3.amazonaws.com/tripdata/"
url_end = "-citibike-tripdata.zip"
url_end_alt = "-citibike-tripdata.csv.zip"
good_columns = [
    "tripduration",
    "starttime",
    "startstationid",
    "endstationid",
    "usertype",
]

In [46]:
# this will download all csvs
for i, ym in enumerate(year_months):
    if not os.path.isfile(f"{ym}.csv.gz"):
        clear_output()
        print(f"getting csv {i+1} of {len(year_months)}, {ym}")
        # there are two different url formats, so we try the first one
        # and fall back to the second if the first returns a 404
        try:
            response = requests.get(url_start + ym + url_end)
            response.raise_for_status()
        except requests.exceptions.HTTPError:
            response = requests.get(url_start + ym + url_end_alt)
        zf = ZipFile(BytesIO(response.content))
        csv = zf.read(zf.namelist()[0]).decode()
        df = pd.read_csv(
            StringIO(csv),
            # the column names change somewhat over the months, so we use
            # this lambda to reduce them to a normalized format
            usecols=lambda x: x.lower().replace(" ", "") in good_columns,
            # use column indices instead of names since the names change slightly
            dtype={3: "Int64", 4: "Int64", 12: "category"},
        )
        # normalize once and for all
        df.columns = [col.lower().replace(" ", "") for col in df.columns]
        # save some space by replacing Subscriber with S and Customer with C
        df.iloc[:, -1].cat.rename_categories(lambda x: x[0], inplace=True)
        df.to_csv(f"{ym}.csv.gz", index=False)
print("done")

getting csv 75 of 75, 201908
done


In [47]:
class MinMaxLedger:
    def __init__(self):
        self.max = defaultdict(lambda: datetime.datetime.min)
        self.min = defaultdict(lambda: datetime.datetime.max)

    def enter_stats(self, station_id, min_start_time, max_start_time):
        if min_start_time < self.min[station_id]:
            self.min[station_id] = min_start_time
        if self.max[station_id] < max_start_time:
            self.max[station_id] = max_start_time

In [48]:
ledger = MinMaxLedger()

for ym in year_months:

    try:
        df = pd.read_csv(
            f"{ym}.csv.gz",
            dtype={
                "tripduration": "Int64",
                "startstationid": "Int64",
                "usertype": "category",
            },
            parse_dates=["starttime"],
            infer_datetime_format=True,
        )
    except FileNotFoundError:
        break
    clear_output()
    print(ym)
    stats = df.groupby("startstationid").agg(
        min_start_time=pd.NamedAgg("starttime", "min"),
        max_start_time=pd.NamedAgg("starttime", "max"),
    )
    stats.apply(
        lambda x: ledger.enter_stats(x.name, x.min_start_time, x.max_start_time), axis=1
    )

201908


In [57]:
with open("ts_stations.pickle", "wb") as f:
    pickle.dump([dict(ledger.min), dict(ledger.max)], f)